In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
%cd /content/drive/MyDrive/OptiML

In [ ]:
! pip install foolbox==3.3.1

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import sklearn
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
from torchvision import datasets, transforms
from sklearn.metrics import accuracy_score, recall_score, f1_score
from Functions.implementations import *
from Functions.visualization import *
import foolbox as fb

In [ ]:
# Define your model architecture (must match the saved model)

set_seed(42)

model_path = "./Results/Adam_VGG/VGG_lr_0.0005_beta1_0.9_beta2_0.99.pth"  # Update with your model path

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = VGGLike().to(device)
model.load_state_dict(torch.load(model_path, map_location=device))
model.eval()

In [ ]:
fmodel = fb.PyTorchModel(model, bounds=(0, 1))

# Sample input and label
train_loader,valid_loader, test_loader = get_data_loaders(batch_size=128)

# Get a single (image, label) pair from the DataLoader
image, label = next(iter(test_loader))  # image: [1, 3, 32, 32], label: [1]
image = image.to(device)
label = label.to(device)

# Run a black-box attack (e.g., Boundary Attack)
attack = fb.attacks.BoundaryAttack()
raw_advs, clipped_advs, success = attack(fmodel, image, label, epsilons=None)

print(success)